In [3]:
import os

import requests
import json
import pandas as pd
import numpy as np
from dotenv import load_dotenv

load_dotenv()
KAMIS_CERT_KEY = os.getenv('KAMIS_CERT_KEY')
KAMIS_CERT_ID = os.getenv('KAMIS_CERT_ID')
CATEGORY_CODES = ['100', '200', '300', '400']

In [2]:
# 18-01-01 ~ 23-12-31
# 사용할 작물별로 그날의 평균값
# 10일 단위로 최대 최소 가격
# 10일 단위 전체 평균 가격

In [21]:
# api 호출
def get_daily_price(date: str, category_code: str) -> json:
  headers = {'Content-Type': 'application/json', 'charset': 'UTF-8', 'Accept': '*/*'}
  request_url = 'http://www.kamis.or.kr/service/price/xml.do?action=dailyPriceByCategoryList'
  params = {
    'p_cert_key':KAMIS_CERT_KEY,
    'p_cert_id':KAMIS_CERT_ID,
    'p_item_category_code':category_code,
    'p_regday':date,
    'p_returntype':'json'
  }
  
  for param in params.keys():
    request_url += f"&{param}={params[param]}"
    
  try:
    return requests.get(request_url, headers=headers).json()
  except Exception as e:
    print(e)

In [14]:
data = pd.DataFrame(columns=['YEAR', 'TENDAYS_COUNT', 'PRODUCT_CODE', 'PRODUCT_NAME', 'MIN_PRODUCT_COST', 'MAX_PRODUCT_COST', 'AVG_PRODUCT_COST'])

In [27]:
date = '2018-01-01'
cat = '100'

response = get_daily_price(date, cat)

In [30]:
for item in response['data']['item']:
  item_code = item['item_code']
  item_name = item['item_name']
  product_cost = item['dpr1']
  
  if product_cost == '-':
    continue

SyntaxError: incomplete input (809658911.py, line 1)

In [31]:
response['data']['item']

[{'item_name': '쌀',
  'item_code': '111',
  'kind_name': '20kg(20kg)',
  'kind_code': '01',
  'rank': '상품',
  'rank_code': '04',
  'unit': '20kg',
  'day1': '당일 (01/01)',
  'dpr1': '-',
  'day2': '1일전 (12/31)',
  'dpr2': '-',
  'day3': '1주일전 (12/25)',
  'dpr3': '-',
  'day4': '2주일전 (12/18)',
  'dpr4': '39,000',
  'day5': '1개월전',
  'dpr5': '38,520',
  'day6': '1년전',
  'dpr6': '32,400',
  'day7': '일평년',
  'dpr7': '40,400'},
 {'item_name': '쌀',
  'item_code': '111',
  'kind_name': '20kg(20kg)',
  'kind_code': '01',
  'rank': '중품',
  'rank_code': '05',
  'unit': '20kg',
  'day1': '당일 (01/01)',
  'dpr1': '-',
  'day2': '1일전 (12/31)',
  'dpr2': '-',
  'day3': '1주일전 (12/25)',
  'dpr3': '-',
  'day4': '2주일전 (12/18)',
  'dpr4': '37,000',
  'day5': '1개월전',
  'dpr5': '36,520',
  'day6': '1년전',
  'dpr6': '30,400',
  'day7': '일평년',
  'dpr7': '39,300'},
 {'item_name': '찹쌀',
  'item_code': '112',
  'kind_name': '일반계(40kg)',
  'kind_code': '01',
  'rank': '상품',
  'rank_code': '04',
  'unit': '40kg',
 